In [8]:
import os
import pickle

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, fbeta_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer


In [23]:
train = pd.read_csv("/content/sample_data/realty_data.csv")

In [24]:
train.head(10)

,product_name,period,price,postcode,address_name,lat,lon,object_type,total_square,rooms,floor,city,settlement,district,area,description,source
0,"3-комнатная, 137 м²",NaN,63000000,127473.0,"2-й Щемиловский переулок, 5а",55.778894,37.608844,Квартира,137.0,3.0,6.0,Москва,NaN,Тверской район,NaN,Просторная квартира свободной планировки с пан...,ЦИАН
1,"Студия, 16,7 м²",NaN,3250000,108815.0,"Харлампиева, 46",55.551025,37.313054,Квартира,16.7,NaN,1.0,Москва,NaN,Филимонковское поселение,NaN,ВНИМАНИЕ! ОЧЕНЬ ПРИВЛЕКАТЕЛЬНОЕ ПРЕ...,Домклик
2,"3-комнатная, 76 м²",NaN,16004680,NaN,"ЖК Прокшино, 8 к4",55.594802,37.431264,Квартира,76.0,3.0,6.0,Москва,NaN,Сосенское поселение,NaN,"Apт.1684018. 0,01% - гибкая ипотека! Воспользу...",Яндекс.Недвижимость
3,"1-комнатная, 24 м²",NaN,7841776,NaN,"ЖК Прокшино, 6 к2",55.594332,37.428099,Квартира,24.0,1.0,10.0,Москва,NaN,Сосенское поселение,NaN,Продается однокомнатная квартира № 381 в новос...,Новострой-М
4,"3-комнатная, 126 м²",NaN,120000000,121352.0,"Давыдковская, 18",55.721097,37.464342,Квартира,126.0,3.0,16.0,Москва,NaN,Фили-Давыдково район,NaN,Шикарное предложение!\nПродаётся трёхкомнатная...,Домклик
5,"3-комнатная, 74 м²",NaN,13300000,142721.0,"Пригородное шоссе, 3",55.557342,37.809447,Квартира,74.0,3.0,14.0,NaN,д. Мисайлово,NaN,NaN,Арт. 32494997 \nОтличная 3-х комнатная квартир...,ЦИАН
6,"2-комнатная, 65 м²",NaN,9500000,143986.0,"Автозаводская, 4 к2",55.740408,38.011392,Квартира,65.0,2.0,3.0,Балашиха,NaN,NaN,Железнодорожный м-н,Предлагается к покупке просторная двухкомнатна...,Домклик
7,"3-комнатная, 87 м²",NaN,15000000,143072.0,"Михаила Кутузова, 15",55.659569,37.218370,Квартира,87.0,3.0,11.0,NaN,пос. ВНИИССОК,NaN,NaN,"ЖК Гусарская Баллада, кирпичный дом, огороженн...",ЦИАН
8,"2-комнатная, 35,8 м²",NaN,7135671,NaN,"3-й квартал, к66",55.700571,37.962692,Квартира,35.8,2.0,11.0,Люберцы,NaN,NaN,Самолёт м-н,Продается двухкомнатная квартира № 542 в новос...,Новострой-М
9,"2-комнатная, 68,3 м²",NaN,31500000,127427.0,"улица Академика Королёва, 21",55.820044,37.602498,Квартира,68.3,2.0,4.0,Москва,NaN,Марфино район,NaN,Дом ''Достижение'' располагается в Москве в ок...,ЦИАН


In [12]:
train.columns.tolist()


['product_name',
 'period',
 'price',
 'postcode',
 'address_name',
 'lat',
 'lon',
 'object_type',
 'total_square',
 'rooms',
 'floor',
 'city',
 'settlement',
 'district',
 'area',
 'description',
 'source']

In [25]:
train = train.drop(columns=['product_name','period', 'postcode', 'address_name', 'lat', 'lon', 'object_type',   'city', 'settlement',
'district', 'area', 'description', 'source']) # уберем столбцы с не самыми важными данными
train

,price,total_square,rooms,floor
0,63000000,137.0,3.0,6.0
1,3250000,16.7,NaN,1.0
2,16004680,76.0,3.0,6.0
3,7841776,24.0,1.0,10.0
4,120000000,126.0,3.0,16.0
...,...,...,...,...
98817,12183528,42.0,2.0,8.0
98818,34925000,63.5,2.0,3.0
98819,10000000,34.4,1.0,3.0
98820,15999000,56.0,3.0,11.0


In [26]:
train.isna().sum()

,0
price,0
total_square,0
rooms,3982
floor,0


In [27]:
train['rooms'] = train['rooms'].fillna(train['rooms'].median()) # заполним пропуски средними занчениями

In [28]:
train.dropna(inplace=True)

In [29]:
train.head(10)

,price,total_square,rooms,floor
0,63000000,137.0,3.0,6.0
1,3250000,16.7,2.0,1.0
2,16004680,76.0,3.0,6.0
3,7841776,24.0,1.0,10.0
4,120000000,126.0,3.0,16.0
5,13300000,74.0,3.0,14.0
6,9500000,65.0,2.0,3.0
7,15000000,87.0,3.0,11.0
8,7135671,35.8,2.0,11.0
9,31500000,68.3,2.0,4.0


In [30]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98822 entries, 0 to 98821
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         98822 non-null  int64  
 1   total_square  98822 non-null  float64
 2   rooms         98822 non-null  float64
 3   floor         98822 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 3.0 MB


In [31]:
train.describe()

,price,total_square,rooms,floor
count,9.882200e+04,98822.000000,98822.000000,98822.000000
mean,2.512122e+07,66.092176,2.189472,9.905274
std,3.607234e+07,48.816204,1.018227,8.219180
min,1.900000e+06,8.000000,1.000000,1.000000
25%,1.050000e+07,40.100000,1.000000,4.000000
50%,1.516713e+07,56.400000,2.000000,8.000000
75%,2.500000e+07,75.700000,3.000000,14.000000
max,1.155219e+09,2070.000000,15.000000,66.000000


In [32]:
X = train[['total_square',	'rooms',	'floor']]
y = train['price']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
# Создание и обучение модели линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [35]:
# Прогнозирование на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 348043247672001.25
R^2 Score: 0.7401465817897848


In [36]:
y_pred

array([17770847.85489515, 13449628.56526122, 31180974.8680256 , ...,
       14352363.18225075,  9529455.85861309, 43405579.05735081])

In [ ]:
with open('rf_fitted.pkl', 'wb') as file:
        pickle.dump(model, file)


def read_model(model_path):
    if not os.path.exists(model_path):
        raise FileNotFoundError("Model file not exists")

    with open(model_path, 'rb') as file:
        model = pickle.load(file)

    return model